In [ ]:
from neurokit.io import Recording
from neurokit.io import read_edf
from neurokit.analysis import suppressions
from neurokit.utils import intervals_to_mask
from neurokit.preprocessing import detect_artifacts
from sklearn.metrics import confusion_matrix
from plotly.subplots import make_subplots
from scipy.ndimage import morphology
import neurokit as nk
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import sklearn
import matplotlib.pyplot as plt

#### Helper functions

In [ ]:
def _calculate_perf_measures(prediction, reference):
    TN, FP, FN, TP = confusion_matrix(reference, prediction).ravel()
    accuracy = (prediction == reference).mean()
    sensitivity = TP/(TP+FN)
    specificity = TN/(TN+FP)
    precision = TP/(TP+FP)
    fnr = FN/(FN+TP)
    fpr = FP/(FP+TN)
    return {'accuracy': accuracy,
            'sensitivity': sensitivity,
            'specificity': specificity,
            'precision': precision,
            'fnr': fnr,
            'fpr': fpr}

def _get_values(values, idx):
    orig = values.copy()
    mask = values.copy()
    orig[idx] = np.nan
    mask[~idx] = np.nan
    return orig, mask

def _plot_suppression(data, pred, ref, channel, sup_type):
    values = data.loc[:, channel]
    ref_ies = ref.loc[:, sup_type].values.astype(bool)
    orig , ies = _get_values(values, ref_ies)
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, subplot_titles=('Reference', 'Prediction'))
    fig.add_trace(go.Scatter(x=data.index, y=orig, line_color="#1f77b4", name=channel), row=1, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=ies, line_color="#d62728", name=sup_type), row=1, col=1)
    o2, ies2 = _get_values(values, pred)
    fig.add_trace(go.Scatter(x=data.index, y=o2, line_color="#1f77b4", name=channel), row=2, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=ies2, line_color="#d62728", name=sup_type), row=2, col=1)
    fig.show()


### Reading segments of eeg 

In [ ]:
seg1 = read_edf('data/EEG_BM65_ies_as.edf')
seg2 = read_edf('data/EEG_BM65_none.edf')
seg3 = read_edf('data/EEG_BM65_ies.edf')
seg4 = read_edf('data/EEG_BM65.edf')

### Reading reference masks calculated using Matlab code

In [ ]:
ref1 = pd.read_csv('data/BM65_ies_as.csv')
ref2 = pd.read_csv('data/BM65_no_sup.csv')
ref3 = pd.read_csv('data/BM65_ies.csv')
ref4 = pd.read_csv('data/BM65_artefact.csv')

### Calculating detections

#### Case 1  : IES detections

#### metrics

In [ ]:
intervals1 = suppressions.detect_ies(seg1, ['EEG L1(Fp1)', 'EEG R1(Fp2)'], min_duration=2.5)
intervals2 = suppressions.detect_ies(seg2, ['EEG L1(Fp1)', 'EEG R1(Fp2)'], min_duration=2.5)
intervals3 = suppressions.detect_ies(seg3, ['EEG L1(Fp1)', 'EEG R1(Fp2)'], min_duration=2.5)
ies_pred1 = intervals_to_mask(intervals1.loc[:,['start', 'end']].values, seg1.data.index)
ies_pred2 = intervals_to_mask(intervals2.loc[:,['start', 'end']].values, seg2.data.index)
ies_pred3 = intervals_to_mask(intervals3.loc[:,['start', 'end']].values, seg3.data.index)
perf = pd.DataFrame(columns=['accuracy','sensitivity','specificity','precision','fnr','fpr'])
perf = perf.append(_calculate_perf_measures(ies_pred1, ref1.loc[:, 'ies']),ignore_index=True)
perf = perf.append(_calculate_perf_measures(ies_pred2, ref2.loc[:, 'ies']),ignore_index=True)
perf = perf.append(_calculate_perf_measures(ies_pred3, ref3.loc[:, 'ies']),ignore_index=True)
perf

#### Case 2 : alpha suppression detection

In [ ]:
intervals1 = suppressions.detect_alpha_suppressions(seg1)
intervals2 = suppressions.detect_alpha_suppressions(seg2)
intervals3 = suppressions.detect_alpha_suppressions(seg3)
as_pred1 = intervals_to_mask(intervals1.loc[:,['start', 'end']].values, seg1.data.index)
as_pred2 = intervals_to_mask(intervals2.loc[:,['start', 'end']].values, seg2.data.index)
as_pred3 = intervals_to_mask(intervals3.loc[:,['start', 'end']].values, seg3.data.index)
perf = pd.DataFrame(columns=['accuracy','sensitivity','specificity','precision','fnr','fpr'])
perf = perf.append(_calculate_perf_measures(as_pred1, ref1.loc[:, 'as']),ignore_index=True)
perf = perf.append(_calculate_perf_measures(as_pred2, ref2.loc[:, 'as']),ignore_index=True)
perf = perf.append(_calculate_perf_measures(as_pred3, ref3.loc[:, 'as']),ignore_index=True)
perf

### Plotting suppressions
#### IES Suppressions
Plotting suppressions separately to analyse performance
##### Case 1
Presence of both IES and Alpha supressions

In [ ]:
_plot_suppression(seg1.data, ies_pred1, ref1, 'EEG R1(Fp2)','ies')

##### Case 2
Ideal case of no suppression, but MATLAB code detects both IES and alpha suppressions

In [ ]:
_plot_suppression(seg2.data, ies_pred2, ref2, 'EEG R1(Fp2)','ies')

##### Case 3
only IES 

In [ ]:
_plot_suppression(seg3.data, ies_pred3, ref3, 'EEG R1(Fp2)','ies')

### Alpha Suppressions
##### Case 1
Both Alpha and IE suppressions

In [ ]:
_plot_suppression(seg1.data, ies_pred1, ref1, 'EEG R1(Fp2)','as')

##### Case 2 
Ideally no IES or Apha suppression, but detected by Matlab code

In [ ]:
_plot_suppression(seg2.data, ies_pred2, ref2, 'EEG R1(Fp2)','as')

##### Case 3
No Alpha suppressions

In [ ]:
_plot_suppression(seg3.data, ies_pred3, ref3, 'EEG R1(Fp2)','as')

### Artifact detection
detecting artifacts using neurokit.preprocessing.artifact

In [ ]:
seg5 = seg4.copy()
seg5.data = seg4.data.loc[:,['EEG R1(Fp2)','EEG L2(F7)']]
intervals4 = detect_artifacts(seg5,detectors={"amplitude"})
artefacts_pred = intervals_to_mask(intervals4.loc[:,['start', 'end']].values, seg4.data.index)

In [ ]:
_plot_suppression(seg4.data, artefacts_pred, ref4, 'EEG R1(Fp2)','artefact')

In [ ]:
morphed = morphology.binary_dilation(artefacts_pred, structure=np.ones(round(seg4.frequency)))
_plot_suppression(seg4.data, morphed, ref4, 'EEG R1(Fp2)','artefact')

In [ ]:
intervals5 = detect_artifacts(seg3,detectors={"amplitude"})
artefacts_pred2 = intervals_to_mask(intervals5.loc[:,['start', 'end']].values, seg3.data.index)
morphed = morphology.binary_dilation(artefacts_pred2, structure=np.ones(round(seg4.frequency)))
_plot_suppression(seg3.data, morphed, ref3, 'EEG R1(Fp2)','artefact')